# Interpolation & Data Preparation
This notebook is to do the interpolation for Accumulated Growing Degree Days (AGDD), by using
   - Inverse distance weighted (IDW)
   - Ordinary Kriging
   - Universal Kriging 
   
and prepare soil moisture, then converting the rasters to points, and saving them into a PostGIS database on Google Cloud.

### Package

In [41]:
import arcpy
import os
import psycopg2
import random

### Workspace

In [43]:
arcpy.env.workspace = r"D:\spring2024\GIS5572\Final\Final project GIS5572.gdb"

### Connect to PostGIS database

In [67]:
conn = psycopg2.connect(
    dbname="gis5572",
    user="postgres",
    password="",
    host="",
    port="5432"
)
cur = conn.cursor()

### Interpolation algorithms results

In [7]:
# ORDINARY_KRIGING
with arcpy.EnvManager(scratchWorkspace=arcpy.env.workspace):
    out_surface_raster = arcpy.sa.Kriging(
        in_point_features="IEM_temp_data",
        z_field="Cumulative_GDD",
        kriging_model="Spherical # # # #",
        cell_size=0.0218500823959998,
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    out_surface_raster.save(arcpy.env.workspace + "\\Kriging_ord")

In [9]:
# UNIVERSAL_KRIGING
with arcpy.EnvManager(scratchWorkspace=arcpy.env.workspace):
    out_surface_raster = arcpy.sa.Kriging(
        in_point_features="IEM_temp_data",
        z_field="Cumulative_GDD",
        kriging_model="LinearDrift 0.021850 # # #",
        cell_size=0.0218500823959998,
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    out_surface_raster.save(arcpy.env.workspace + "\\Kriging_uni")

In [10]:
# IDW
with arcpy.EnvManager(scratchWorkspace=arcpy.env.workspace):
    out_raster = arcpy.sa.Idw(
        in_point_features="IEM_temp_data",
        z_field="Cumulative_GDD",
        cell_size=0.0218500823959998,
        power=2,
        search_radius="VARIABLE 12",
        in_barrier_polyline_features=None
    )
    out_raster.save(arcpy.env.workspace + "\\Idw")

### Soil Moisture
Soil Moisture data was download from NASA SMAP by Google Earth Engine



In [7]:
SM = "Soil Moisture MN.tif"

### Saves the rasters results to PostGIS
In this process, several steps are involved:

1. Converting the raster to points.
2. Clipping the points using the Minnesota boundary.
3. Employing Bernoulli sampling to sample the points.
    - Due to the extensive dataset size, uploading to AGOL would be impacted.
4. Saving the final points into a PostGIS database.

In [17]:
def bernoulli_sampling(input_point_data, sampling_ratio):
    # iterate through the points and apply Bernoulli sampling
    with arcpy.da.UpdateCursor(input_point_data, ["OID@"]) as cursor:
        for row in cursor:
            # generate a random number between 0 and 1
            random_value = random.random()
            # if the random value is greater than or equal to the sampling ratio, delete the point
            if random_value >= sampling_ratio:
                cursor.deleteRow()

def raster_pts_SDE(input_raster, output_pt_name, sampling_ratio):
    # convert raster to point
    arcpy.conversion.RasterToPoint(
        in_raster=input_raster,
        out_point_features=output_pt_name,
        raster_field="Value")
    
    # clip the data by MN boundary
    clipped_data = arcpy.analysis.Clip(
        in_features=output_pt_name,
        clip_features="state_of_minnesota",
        out_feature_class=f"samp_{output_pt_name}",
        cluster_tolerance=None
    )

    # perform Bernoulli sampling on the clipped point data
    bernoulli_sampling(clipped_data, sampling_ratio)
    
    # save data to PostGIS    
    arcpy.conversion.FeatureClassToGeodatabase(
        Input_Features= clipped_data,
        Output_Geodatabase="\\PostgreSQL-34-gis5572(postgres).sde")
    
    print(f"{input_raster} has been converted to points and stored into SDE as {os.path.basename(clipped_data.getOutput(0))}.")



In [18]:
# execute the function
rasters = ["Idw", "Kriging_ord", "Kriging_uni"]

for raster in rasters:

    input_raster = raster
    output_pt_name = f"AGDD_{raster}"
    sampling_ratio = 0.1  
    raster_pts_SDE(input_raster, output_pt_name, sampling_ratio)

Idw has been converted to points and stored into SDE as samp_AGDD_Idw.
Kriging_ord has been converted to points and stored into SDE as samp_AGDD_Kriging_ord.
Kriging_uni has been converted to points and stored into SDE as samp_AGDD_Kriging_uni.


In [19]:
# soil moisture 
input_raster = "Soil Moisture MN.tif"
output_pt_name = "Soil_Moisture"
sampling_ratio = 0.1  

raster_pts_SDE(input_raster, output_pt_name, sampling_ratio)

Soil Moisture MN.tif has been converted to points and stored into SDE as samp_Soil_Moisture.
